In [1]:
require 'nn'
require 'hdf5'
require 'optim'

In [2]:
myFile = hdf5.open('6-grams.hdf5','r')
data = myFile:all()
myFile:close()

train = data['train_nocounts']:narrow(2,1,6)
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

valid_topredict = data['valid']:narrow(2,1,50)
valid_input = data['valid']:narrow(2,51,5)
valid_output = data['valid_output']

In [3]:
nwords = data['nwords'][1]

N = 6
dwin = N-1
hid1 = 30.
hid2 = 100

nnlm = nn.Sequential()

LT = nn.LookupTable(nwords,hid1)

nnlm:add(LT)
nnlm:add(nn.View(1,-1,hid1*dwin))
nnlm:add(nn.Squeeze())
nnlm:add(nn.Linear(hid1*dwin,hid2))
nnlm:add(nn.Tanh())
nnlm:add(nn.Linear(hid2, nwords))
nnlm:add(nn.LogSoftMax())

In [4]:
criterion = nn.ClassNLLCriterion()

In [16]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train_input[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end

In [20]:
for i = 1, 1 do
    
    timer = torch.Timer()
    LT.weight:renorm(2,1,1)
    trainer = nn.StochasticGradient(nnlm, criterion)
    trainer.learningRate = 0.005
    trainer.maxIteration = 1
    trainer:train(dataset)
    print('Epoch '..i..': '..timer:time().real)
    
end

# StochasticGradient: training	


# current error = 6.0455673012034	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 6.0455673012034	
Epoch 1: 2320.9671299458	


In [24]:
kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.zeros(valid_input:size(1),50)

for i = 1, valid_input:size(1) do
    kag_pred_valid[i]:copy(nnlm:forward(valid_input[i]):index(1, valid_topredict[i])):exp()
end

norm_mat:zero()
norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat)

CE = 0
for i = 1, valid_input:size(1) do
    mm,aa = valid_output[i]:max(1)
    CE = CE + math.log(kag_pred_valid[i][aa[1]])
end

val_res = math.exp(-CE/kag_pred_valid:size(1))
print('Perplexity on valid: '..val_res)

Perplexity on valid: 5.637808509993	


## using optim:

In [5]:
parameters, gradParameters = nnlm:getParameters()

In [10]:
batchSize = 32
kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.zeros(valid_input:size(1),50)

for i = 1,1  do
    
    timer = torch.Timer()
    
    LT.weight:renorm(2,1,1)
    
    av_L = 0
--     for t = 1, train_input:size(1), batchSize do
    for t = 1, train_input:size(1), batchSize do
      -- create mini batch
        inputs = torch.Tensor(batchSize,dwin)
        targets = torch.Tensor(batchSize)
        k = 1
        for j = t,math.min(t+batchSize-1,train_input:size(1)) do
         -- load new sample
            inputs[k] = train_input[j]
            targets[k] = train_output[j]
            k = k + 1
        end
        -- reset gradients
        gradParameters:zero()

        -- evaluate function for complete mini batch
        outputs = nnlm:forward(inputs:narrow(1,1,k-1))
        f = criterion:forward(outputs:narrow(1,1,k-1), targets:narrow(1,1,k-1))
        av_L = av_L +f
        df_do = criterion:backward(outputs:narrow(1,1,k-1), targets:narrow(1,1,k-1))
        nnlm:backward(inputs:narrow(1,1,k-1), df_do)
        nnlm:updateParameters(0.01)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Loss: '..av_L/math.floor(train_input:size(1)/batchSize))
    
    -- Evaluatin perplexity on validation:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[i]:copy(nnlm:forward(valid_input[ii]):index(1, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

1	


33	


65	


97	


129	


...

52353	


52385	


52417	


52449	


52481	


In [8]:
train_input:size(1)

887522	


### NCE:

In [ ]:
N = 6
dwin = N-1
hid1 = 30.
hid2 = 100

nnlm = nn.Sequential()

LT = nn.LookupTable(nwords,hid1)

nnlm:add(LT)
nnlm:add(nn.View(1,-1,hid1*dwin))
nnlm:add(nn.Squeeze())
nnlm:add(nn.Linear(hid1*dwin,hid2))
nnlm:add(nn.Tanh())

final_lin =  nn.Linear(hid2, nwords)

nnlm:add(final_lin)

nnlm:add(nn.LogSoftMax())


nce = nn.Sequential()
